In [5]:
# !sudo yum install Xvfb -y -y
# !pip install scrapy
# !pip install fake_useragent
# !pip install scrapy-selenium
# !pip install selenium==4.9.1
# !pip install boto3==1.28.5
# !pip install crochet
# !pip install awslambdaric
# !pip install botocore==1.31.10
# #

# import os
# cmd = '''
# sudo yum install atk cups-libs gtk3 libXcomposite alsa-lib libXcursor libXdamage libXext libXi libXrandr libXScrnSaver libXtst pango at-spi2-atk libXt xorg-x11-server-Xvfb xorg-x11-xauth dbus-glib dbus-glib-devel -y
# '''
# os.system(cmd)


# !curl  -Lo "/tmp/chromedriver.zip" "https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip"
# !unzip /tmp/chromedriver.zip -d /opt/ -y
# !sudo ln -sf /opt/chromedriver /usr/local/bin/chromedriver

# !curl -Lo "/tmp/chrome-linux.zip" "https://www.googleapis.com/download/storage/v1/b/chromium-browser-snapshots/o/Linux_x64%2F1135561%2Fchrome-linux.zip?alt=media"
# !unzip /tmp/chrome-linux.zip -d /opt/ -y
# !sudo ln -sf /opt/chrome-linux/chrome /usr/local/bin/chrome




In [29]:
import datetime
import json
import os
import botocore
from fake_useragent import UserAgent
import boto3
from scrapy.crawler import CrawlerRunner
from scrapy.utils.project import get_project_settings
from crochet import setup
import threading
# from momoshop_link import MomoshopSpider

# Initialize Crochet
setup()

ua = UserAgent()
region = 'ap-northeast-1'
sqs = boto3.client('sqs', region_name=region, config=botocore.client.Config(max_pool_connections=500))
# queue_tier4_links = 'https://sqs.ap-northeast-1.amazonaws.com/407620147666/aws_crawler_tier4_links.fifo'

In [68]:
import datetime
import hashlib
import json
import os
import threading
from urllib.parse import urlparse

from scrapy import Selector
from scrapy_selenium import SeleniumRequest
import botocore
import scrapy
from fake_useragent import UserAgent
import boto3
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec

ua = UserAgent()
# sqs = boto3.client('sqs', region_name=region, config=botocore.client.Config(max_pool_connections=500))
# s3 = boto3.client('s3', region_name=region, config=botocore.client.Config(max_pool_connections=500))
region = 'ap-northeast-1'
# export_to_s3 = os.getenv('EXPORT_TO_S3')
queue_tier_content_links =  'https://sqs.ap-northeast-1.amazonaws.com/407620147666/aws_crawler_tier4_links.fifo'



def hash_function(original_string):
    # Create a hash object using SHA-256 algorithm
    hasher = hashlib.sha256()

    # Convert the input string to bytes (required by hashlib)
    input_bytes = original_string.encode('utf-8')

    # Update the hash object with the input bytes
    hasher.update(input_bytes)

    # Get the hexadecimal representation of the hash value
    return hasher.hexdigest()

# add on 08/30
def string_clean(text: str):
    text = text.replace('\n', '').replace('\t', '').replace(',', '').replace('\r', '').replace(' ', '').replace('$', '').replace('\xa0', '')
    return text 

# add on 08/30
def CJK_cleaner(string):
    import re
    filters = re.compile('[^0-9a-zA-Z\\u4e00-\\u9fff]+', re.UNICODE)
    return filters.sub(' ', string)

class IOpenMallSpider(scrapy.Spider):
    name = "iopenmall" # modify
    allowed_domains = ["mall.iopenmall.tw/iopen/"]  # modify
    start_urls = "https://mall.iopenmall.tw/iopen/" # modify
    user_agent = ua.random
    runner = None
    category_links = []
    tier = None

    def __init__(self, runner=None, **kwargs):
        self.runner = runner
        super(IOpenMallSpider, self).__init__(**kwargs) # modify

    def start_requests(self):
        # Start the initial request to fetch category links
        print("Starting start_requests")
        
        if self.runner.target_url == "":
            tier_content_link = json.loads(self.runner.tier4_content_object['Body'])['tier_content_link']
        else:
            tier_content_link = self.runner.target_url

        headers = {"Host": urlparse(tier_content_link).netloc,
                   'Accept-Encoding': 'gzip, deflate, br',
                   'Accept-Language': 'en-US,en;q=0.9,zh-TW;q=0.8,zh;q=0.7,zh-CN;q=0.6,ja;q=0.5',
                   'Sec-Fetch-Dest': 'document',
                   'Sec-Fetch-Mode': 'navigate',
                   'Sec-Fetch-Site': 'none',
                   'Upgrade-Insecure-Requests': '1',
                   "Referer": "https://mall.iopenmall.tw/iopen/",
                   "User-Agent": ua.random}
        print(f'tier_content_link: {tier_content_link}')

        yield SeleniumRequest(url=tier_content_link,
                              headers=headers,
                              callback=self.parse,
                              wait_time=15,
                              errback=self.error_handle,
                              wait_until=ec.presence_of_element_located((By.CSS_SELECTOR, '.first.selected'))
                              )

    # modify on 08/30
    @staticmethod
    def set_all_tier(self, tier: dict):
        tier['tier1'] = self.runner.tier1
        tier['tier2'] = self.runner.tier2
        tier['tier3'] = self.runner.tier3
        tier['tier4'] = self.runner.tier4 if self.runner.category_type == 'tier4' else ''
        tier['tier5'] = self.runner.tier5 if self.runner.category_type == 'tier5' else ''
        tier['category_type'] = self.runner.category_type
        print(tier)

    @staticmethod
    def construct_s3_tier_folder(content_tier: dict):
        # Construct the base folder path
        base_path = "content"

        # Create the base folder if it doesn't exist
        if base_path:
            s3.put_object(Bucket=s3_crawler_content_folder, Key=f"{base_path}/")

        # Construct the folder structure based on the tiers
        for key, value in content_tier.items():

            folder_name = value.strip('/')
            folder_path = f"{base_path}/{folder_name}"
            print(folder_path)
            # Create the folder if it doesn't exist
            # s3.put_object(Bucket=s3_crawler_content_folder, Key=f"{folder_path}/")

            # Set the current folder path as the base path for the next iteration
            # base_path = folder_path

    def error_handle(self, error):
        print(error)
        self.runner.timeout = True

    def parse(self, response, **kwargs):
        try:
            # Parse the main category page and extract links to individual product pages
            print(f"Starting to parse")
            selector = Selector(response)
            
            # driver = response.request.meta['driver']
            options = webdriver.ChromeOptions()
            options.add_argument("--headless")
            options.add_argument("--no-sandbox")
            options.add_argument("--single-process")
            options.add_argument("--disable-dev-shm-usage")
            options.add_argument("blink-settings=imagesEnabled=false")
            options.add_argument("--remote-debugging-port=9222")
            options.add_argument("--no-zygote")
            options.add_argument("--disable-gpu")
            options.add_argument("--disable-dev-tools")
    
            driver = webdriver.Chrome(options=options)
            driver.get(response.url)
            content_info_tier = {}                     # modify on 08/30
            self.set_all_tier(self, content_info_tier) # modify on 08/30
            # self.construct_s3_tier_folder(content_info_tier)
            
            # 品牌名稱
            item_brand = selector.css("div.header_logo ::attr(title)").get(None) # modify
            
            # 促銷價
            now_price = None

            # 品牌名稱
            past_price = None

            # 折扣後價格 
            price_list = selector.css(".Product_price ::attr(class)").getall()  # modify
            
            # modify
            if any("now_price" in s for s in price_list) :
                now_price = string_clean(selector.css(".now_price ::text").getall()[-1])
                now_price = int(now_price)

            # modify
            if any("was_price" in s for s in price_list) :
                past_price = selector.css("span.was_price ::text").getall()[-1]
                past_price = string_clean(past_price)
                if past_price != '':
                    past_price = int(past_price)


            # 商品編號
            item_no = selector.css("div.Item_number span ::text").getall()[1] # modify
  
            # 商品名稱
            item_name = string_clean(selector.css("h1.Product_name ::text").get()) # modify
            
            # 商品介紹
            item_note = "//".join(selector.css("div.Product_notes p::text").extract()) # modify
            
            # 付款方式
            pay_method  = string_clean(selector.css("#pay_method ::text").get()) # modify on 08/30
            
            # 運送方式
            delivery_method  = string_clean(selector.css("#delivery_method ::text").get()) # modify
            
            # 活動 # modify
            activity = selector.css(".discountbox ul li ul li ::text").getall()
            for i in range(activity.count('折價券')):
                activity.remove('折價券') 

            # 商品特色
            item_feature = CJK_cleaner("//".join(selector.css('#content1 p ::text').extract())) # modify
            
            # 商品評價 # modify
            if selector.css(".comment_total_amount_num ::text").get() is not None:
                goods_commend = {"count": int(selector.css(".comment_total_amount_num ::text").get())}
            else:
                goods_commend = {"count": 0}

                
            # 評論內容 # modify
            review_card_list = []
            # If there's any goods commend then fetch all of them
            if goods_commend["count"] > 0:
                goods_commend["indicator_average_value"] = float(selector.css('.comment_total_grade_num ::text').get())
                # Fetch the all comments in the first page
                self.fetch_goods_commend(selector, review_card_list)
            
            # 店家資訊 # modify
            store_seller_list = []
            self.fetch_seller(selector, store_seller_list)
            store_info = store_seller_list

            # modify
            content_info = {'item_brand': item_brand,
                            'item_no': item_no,
                            'item_name': item_name,
                            'item_note': item_note,
                            'now_price': now_price,
                            'past_price': past_price,
                            'activity': activity,
                            'payway' : pay_method,
                            'deliveryway' : delivery_method, 
                            'item_feature': item_feature,
                            'store_seller': store_info,
                            'goods_commend': goods_commend,
                            'goods_commend_review_card_list': review_card_list,
                            'data_date': datetime.datetime.now().strftime("%Y%m%d")
                            }
            print(content_info)
            
            with open('output.json', 'w') as json_file:
                json_file.write(json.dumps(content_info, ensure_ascii=False))
        except Exception as e:
            print(str(e))
            
    # modify
    @staticmethod
    def fetch_goods_commend(selector, review_card_list):
        for review_card in selector.css('.comment_data'):
            review_card_score_temp = review_card.css(".comment_grade_star img ::attr(src)").get()
            review_card_score = int((review_card_score_temp.rsplit('_')[-1]).rsplit('.')[0])
            review_card_comment = (review_card.css(".customer_comment_reply p ::text").get())
            if review_card_comment is not None: 
                review_card_comment = string_clean(review_card_comment)
 
            review_card_list.append({
                'review_card_score': review_card_score,
                'review_card_comment': review_card_comment
            })
    
    # add 
    @staticmethod
    def fetch_seller(selector, store_seller_list):
        seller_info_card = selector.css("div.iopen-footer-spec ul.header_spec_v2 li")
        for seller_card in seller_info_card:
            
            item = (seller_card.css("::text").get()).rsplit(':')[0]
            info = str(seller_card.css("span ::text").get()).replace('\xa0', '')
            store_seller_list.append({
                item: info,
            })


    def start_to_push(self, tier_content_links, threads):
        # For each product link, create a message and push it to SQS and/or S3
        for tier_content_link in tier_content_links:
            hash_id = hash_function(tier_content_link)
            message = {'Id': hash_id, 'MessageGroupId': hash_id,
                       'MessageBody': tier_content_link}

#             thread_sqs = threading.Thread(target=self.send_to_sqs, args=(message,))
#             threads.append(thread_sqs)
#             thread_sqs.start()

#             # self.send_to_sqs(message)

#             if export_to_s3 is not None and export_to_s3 == "On":
#                 message_body = {'hash_id': hash_id, 'page_url': tier_content_link}
#                 self.send_to_s3(message_body, hash_id)
#                 thread_s3 = threading.Thread(target=self.send_to_sqs, args=(message,))
#                 threads.append(thread_s3)
#                 thread_s3.start()

#     @staticmethod
#     def send_to_sqs(message):
#         # Send the message to SQS
#         try:
#             # print(message['MessageGroupId'])
#             response = sqs.send_message(QueueUrl=queue_tier_content_links,
#                                         MessageGroupId=message['MessageGroupId'],
#                                         MessageBody=message['MessageBody'],
#                                         MessageDeduplicationId=message['MessageGroupId'])
#             if response["ResponseMetadata"]["HTTPStatusCode"] != 200:
#                 print('Fail')
#         except Exception as e:
#             print(str(e))

#     @staticmethod
#     def send_to_s3(m, hash_id):
#         # Send the message to S3
#         try:
#             json_file_name = f'{hash_id}.json'
#             response = s3.put_object(Bucket=s3_tier_content_links, Key=json_file_name,
#                                      Body=json.dumps(m, ensure_ascii=False))
#             # check if it's successful
#             if response["ResponseMetadata"]["HTTPStatusCode"] != 200:
#                 print('Fail')
#         except Exception as e:
#             print(str(e))

    def spider_closed(self):
        # Handle the spider closing event
        self.runner.timeout = False

In [69]:
class LambdaRunner:
    target_url = ""
    receipt_handle = ""
    tier4_content_object = None
    timeout = False

    def __init__(self, url):
        self.finished = threading.Event()
        self.results = []
        if url != "":
            self.target_url = url

    def run_spider(self):
        # Create a CrawlerRunner with project settings
        settings = get_project_settings()
        runner = CrawlerRunner(settings)
        if self.target_url == "":
            self.get_tier4_content_url_from_sqs()

        # Callback function to handle the spider results
        def handle_results(result):
            self.results.append(result)

            # Check if the spider has finished running
            if len(self.results) == 1:
                self.finished.set()

        # Start the first spider run
        deferred = runner.crawl(IOpenMallSpider, self) #modify
        deferred.addCallback(handle_results)

        # Start the reactor
        runner.join()

    def wait_for_completion(self):
        self.finished.wait()

    def get_results(self):
        return self.results

    def get_tier4_content_url_from_sqs(self):
        response = sqs.receive_message(
            QueueUrl=queue_tier_content_links,
            MaxNumberOfMessages=1,  # Retrieve 10 messages
            WaitTimeSeconds=0  # Maximum time to wait for messages (long polling)
        )
        messages = response.get('Messages', [])
        if messages is not None:
            self.tier4_content_object = messages[0]
        else:
            msg = "All consumed."
            print(msg)
            return msg

        # Delete messages from SQS
        for message in messages:
            sqs.delete_message(
                QueueUrl=queue_tier_content_links,
                ReceiptHandle=message['ReceiptHandle']
            )


def handler(event, context):
    try:
        # Check if the function was triggered by an HTTP request or Lambda event
        print(f"Starting to crawl:{datetime.datetime.now()}")
        times = 0
        if "statusCode" not in event:
            # If the function was not triggered by retry
            runner = LambdaRunner("")
            runner.tier1 = event["tier"]["tier1"]    # add on 08/30
            runner.tier2 = event["tier"]["tier2"]    # add on 08/30 
            runner.tier3 = event["tier"]["tier3"]    # add on 08/30
            runner.tier4 = event["tier"]["tier4"] if event["tier"]["category_type"] == 'tier4' else None   # add on 08/30
            runner.tier5 = event["tier"]["tier5"] if event["tier"]["category_type"] == 'tier5' else None   # add on 08/30
            runner.category_type = event["tier"]["category_type"]  # add on 08/30
            runner.run_spider()
            runner.wait_for_completion()
            print(f"End date and time: {datetime.datetime.now()}")
        else:
            times = int(event["times"])
            if times < 4:
                runner = LambdaRunner(event["tier_content_link"])
                runner.input_url = event["tier_content_link"]
                runner.tier1 = event["tier"]["tier1"]    # add on 08/30
                runner.tier2 = event["tier"]["tier2"]    # add on 08/30
                runner.tier3 = event["tier"]["tier3"]    # add on 08/30
                runner.tier4 = event["tier"]["tier4"] if event["tier"]["category_type"] == 'tier4' else None   # add on 08/30
                runner.tier5 = event["tier"]["tier5"] if event["tier"]["category_type"] == 'tier5' else None   # add on 08/30
                runner.category_type = event["tier"]["category_type"]  # add on 08/30
                runner.run_spider()
                runner.wait_for_completion()
                print(f"End date and time: {datetime.datetime.now()}")
            else:
                print(f'Retry too many times, 429: {event["tier_content_link"]}')
                # If the retry count is 4 or more, return an HTTP 429 response indicating Too Many Requests
                return {
                    'statusCode': 429,
                    'body': "",
                    'times': times,
                    "category_link": event["tier_content_link"]
                }

        times = times + 1
        if not runner.timeout:
            print("success")
            # If the LambdaRunner completed successfully, return an HTTP 200 response with the completion details
            return {
                'statusCode': 200,
                'body': 'Completed!',
                'times': times
            }
        else:
            print('timeout')
            # If the LambdaRunner timed out, return an HTTP 408 response with the category objects
            return {
                'statusCode': 408,
                'times': times,
                "category_link": json.loads(runner.tier4_content_object['Body'])['tier_content_link']
            }
    except Exception as e:
        print(f'fail:{e}')
        return {
            'statusCode': 500,
            'body': json.dumps({'error': str(e)})
        }

# #
# if __name__ == '__main__':
#     handler("", "")

In [70]:
handler({'statusCode': 200,'times': 1, 
    "tier_content_link": "https://mall.iopenmall.tw/016725/index.php?action=product_detail&prod_no=P1672501667466", 
    "tier": {"tier1": "流行時尚", "tier2": "女鞋", "tier3": "跟鞋", "tier4": '', "tier5": '', "category_type": "tier3"}}, "")

Starting to crawl:2023-08-30 07:58:23.372268
Starting start_requests
tier_content_link: https://mall.iopenmall.tw/016725/index.php?action=product_detail&prod_no=P1672501667466
Starting to parse
{'tier1': '流行時尚', 'tier2': '女鞋', 'tier3': '跟鞋', 'tier4': '', 'tier5': '', 'category_type': 'tier3'}
{'item_brand': '韓版女裝．女鞋', 'item_no': 'P1672501667466', 'item_name': '【leleshop】36-40小皮鞋高跟涼拖鞋素色粗跟高跟鞋瑪莉珍鞋懶人鞋牛津鞋方頭高跟鞋#3286', 'item_note': '🎀粗跟小皮鞋 #3268//🔶顏色：黑色、米色、豆沙色//🔸底厚度8公分//🔶尺寸：//36（適合腳長23）//37（適合腳長23.5）//38（適合腳長24）//39（適合腳長24.5）//40（適合腳長25）//🌸版型：正常版//⭕️超甜價$550❤️', 'now_price': 550, 'past_price': '', 'activity': ['0829-0830-滿$399現折$40'], 'payway': '超商取貨付款', 'deliveryway': '常溫7-ELEVEN店到店取貨付款', 'item_feature': ' 粗跟小皮鞋 3268 每個顏色都超美的啦 版媽建議可以直接包色輪流替換 顏色 黑色 米色 豆沙色 底厚度8公分 尺寸 36 適合腳長23 37 適合腳長23 5 38 適合腳長24 39 適合腳長24 5 40 適合腳長25 版型 正常版 超甜價 550 每台顯示器皆不同 依實品顏色為主 高標完美主義者請繞道而行 下單前請注意 現貨 預購 設定7天出貨只是擔心蝦皮 3天後自動取消訂單 商品基本上2 3天就可以收到 如果需要訂購鞋子請等 3 15天 怕等待的水水可先詢問在下單 對我們的商品和服務滿意的 麻煩您給五星好評哦 有不滿意的地方麻煩您在聊聊上面聯繫我們 ', 'stor

{'statusCode': 200, 'body': 'Completed!', 'times': 2}